In [1]:
import pandas as pd
import requests, json
from datetime import date, timedelta
from collections import defaultdict
import xml.etree.ElementTree as ET
from math import dist

In [2]:
KEY = 'XFvzJWgUESMIABoVtfrlH/5UAPe0XlfdQL4aWwGW3PRP7nglPl6l5gbtu9YikPPPpECveIjH0YwuXi9ZccGTcA=='
url = 'http://openapi.forest.go.kr/openapi/service/forestDisasterService/frstFireOpenAPI'
params ={'serviceKey' : KEY, 'searchStDt' : '19841201', 'searchEdDt' : '19841204', 'pageNo' : '1', 'numOfRows' : '10' }

response = requests.get(url, params=params)
print(response.content)

b'<?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><header><resultCode>00</resultCode><resultMsg>NORMAL SERVICE.</resultMsg></header><body><items><item><diravg>W</diravg><dirmax> </dirmax><dmgarea>0.01</dmgarea><dmgmoney>0</dmgmoney><exintgtm>25</exintgtm><extingdt>198412031215</extingdt><humidcurr>0.0</humidcurr><humidmin>0.0</humidmin><humidrel>43.0</humidrel><ocurcause>\xec\x9e\x85\xec\x82\xb0\xec\x9e\x90\xec\x8b\xa4\xed\x99\x94</ocurcause><ocurdo>\xeb\xb6\x80\xec\x82\xb0</ocurdo><ocurdt>198412031150</ocurdt><ocuremd>\xeb\xaa\x85\xeb\xa5\x9c</ocuremd><ocurgm>\xeb\xb6\x80\xec\x82\xb0</ocurgm><ocurjibun>\xec\x82\xb050-7</ocurjibun><ocurri> </ocurri><ocursgg>\xeb\x8f\x99\xeb\x9e\x98</ocursgg><ocuryoil>\xec\x9b\x94</ocuryoil><ownersec>\xec\x82\xac</ownersec><rainamount>2.7</rainamount><raindays>10</raindays><riskavg>0.0</riskavg><riskmax>0.0</riskmax><tempavg>13.4</tempavg><windavg>3.3</windavg><windmax>0.0</windmax></item><item><diravg>NNW</diravg><dirmax> </dirmax><dmg

In [5]:
# 기상데이터 산불발생 시간에 맞춰서 모으고 mongodb 적재하기
# 먼저 산불발생 csv 가져오고
df = pd.read_csv('wildfire_his.csv')
lat_df = pd.read_csv('latitude.csv')
df

,Unnamed: 0,_id,발생일시,발생일시(요일),진화일시(년월일시분),진화소요시간(분),발생장소(관서),발생장소(시도),발생장소(시군구),발생장소(읍면동),...,평균기온,실효습도,상대습도,최소습도,최대풍속,평균풍속,최대풍향,평균풍향,강우결과일수,최종강우량
0,0,62be5e0852482b6dab131e1d,201406191500,목,201406191800,300,북부,강원,인제,남,...,23.6,0.0,57.0,0.0,5.6,1.9,S,SE,6,0.5
1,1,62be5e0852482b6dab131e1e,201406181805,수,201406182015,210,경남,경남,밀양,삼랑진,...,29.6,0.0,46.0,0.0,4.0,2.4,SE,NNE,6,1.5
2,2,62be5e0852482b6dab131e1f,201406151410,일,201406151720,310,강원,강원,인제,인제,...,29.3,0.0,57.0,0.0,6.7,1.3,SSW,NW,2,0.5
3,3,62be5e0852482b6dab131e20,201406141720,토,201406141900,140,전북,전북,부안,보안,...,25.3,0.0,46.0,0.0,7.8,3.3,W,WNW,3,1.5
4,4,62be5e0852482b6dab131e21,201406101718,화,201406102000,242,북부,강원,원주,흥업,...,22.3,0.0,40.0,0.0,9.9,2.0,SSE,NNE,1,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11719,11719,62be5e0852482b6dab134be4,191107022013,,,,대전,대전,중,대사,...,0.0,0.0,0.0,0.0,0.0,0.0,,,0,0.0
11720,11720,62be5e0852482b6dab134be5,191107022013,월,191101010300,1225,대전,대전,중,대사,...,0.0,0.0,0.0,0.0,0.0,0.0,,,0,0.0
11721,11721,62be5e0852482b6dab134be6,191106290400,,,,대전,대전,동,천,...,0.0,0.0,0.0,0.0,0.0,0.0,,,0,0.0
11722,11722,62be5e0852482b6dab134be7,191106290400,월,191101010000,1225,대전,대전,동,천,...,0.0,0.0,0.0,0.0,0.0,0.0,,,0,0.0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11724 entries, 0 to 11723
Data columns (total 28 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   11724 non-null  int64  
 1   _id          11724 non-null  object 
 2   발생일시         11724 non-null  int64  
 3   발생일시(요일)     11724 non-null  object 
 4   진화일시(년월일시분)  11724 non-null  object 
 5   진화소요시간(분)    11724 non-null  object 
 6   발생장소(관서)     11724 non-null  object 
 7   발생장소(시도)     11724 non-null  object 
 8   발생장소(시군구)    11724 non-null  object 
 9   발생장소(읍면동)    11724 non-null  object 
 10  발생장소(리)      11724 non-null  object 
 11  발생장소(지번)     11724 non-null  object 
 12  소유구분         11724 non-null  object 
 13  발생세부원인       11724 non-null  object 
 14  피해면적         11724 non-null  float64
 15  피해액          11724 non-null  int64  
 16  산불위험지수(평균)   11724 non-null  float64
 17  산불위험지수(최대)   11724 non-null  float64
 18  평균기온         11724 non-null  float64
 19  실효습도

In [7]:
df_tm_place=df[["발생일시","발생장소(시도)", "발생장소(시군구)"]]
df_tm_place

,발생일시,발생장소(시도),발생장소(시군구)
0,201406191500,강원,인제
1,201406181805,경남,밀양
2,201406151410,강원,인제
3,201406141720,전북,부안
4,201406101718,강원,원주
...,...,...,...
11719,191107022013,대전,중
11720,191107022013,대전,중
11721,191106290400,대전,동
11722,191106290400,대전,동


In [5]:
lat_df

,Unnamed: 0,_id,지점번호,지점명,위도,경도
0,0,62be6fb4a82199005252108d,90,속초,38.25085,128.56473
1,1,62be6fb4a82199005252108e,93,북춘천,37.94738,127.75443
2,2,62be6fb4a82199005252108f,95,철원,38.14787,127.30420
3,3,62be6fb4a821990052521090,98,동두천,37.90188,127.06070
4,4,62be6fb4a821990052521091,99,파주,37.88589,126.76648
...,...,...,...,...,...,...
90,90,62be6fb4a8219900525210e7,285,합천,35.56505,128.16994
91,91,62be6fb4a8219900525210e8,288,밀양,35.49147,128.74413
92,92,62be6fb4a8219900525210e9,289,산청,35.41300,127.87910
93,93,62be6fb4a8219900525210ea,294,거제,34.88818,128.60458


In [35]:
KAKAO_API_KEY = "da33e829f9b3cce36b4dd72eac9312d6"
KAKAO_MAP_URL = "https://dapi.kakao.com/v2/local/search/address.json?query="

In [36]:
def get_lat(place):
    headers = {
        "Authorization" : "KakaoAK {}".format(KAKAO_API_KEY)
    }
    returned_json=json.loads(str(requests.get(KAKAO_MAP_URL+place,headers=headers).text))
    address = returned_json['documents'][0]['address']
    lat_d = {"lat" : str(address['y']), "lng" : str(address['x'])}
    return lat_d

In [16]:
# 중복제거
df_tm_place=df_tm_place.drop_duplicates()

In [25]:
# 지점번호 리스트 가져오기
point_dict={}
lat_df['지점명']=lat_df['지점명'].map(lambda x: x[:2] if len(x)>2 and (x[2]=='시' or x[2]=='군') else x)
lat_df=lat_df[["지점번호", "지점명","위도","경도"]]
for i in range(len(lat_df)):
    point_dict[lat_df.iloc[i][1]] = [lat_df.iloc[i][0], lat_df.iloc[i][2],lat_df.iloc[i][3]]
    
# list(lat_df["지점명"])
# #시군구 제거
# point_list=list(lat_df['지점명'].map(lambda x: x[:2] if len(x)>2 and (x[2]=='시' or x[2]=='군') else x))
point_dict


{'속초': [90, 38.25085, 128.56473],
 '북춘천': [93, 37.94738, 127.75443],
 '철원': [95, 38.14787, 127.3042],
 '동두천': [98, 37.90188, 127.0607],
 '파주': [99, 37.88589, 126.76648],
 '대관령': [100, 37.67713, 128.71834],
 '춘천': [101, 37.90262, 127.7357],
 '백령도': [102, 37.97396, 124.71237],
 '북강릉': [104, 37.80456, 128.85535],
 '강릉': [105, 37.75147, 128.89099],
 '동해': [106, 37.50709, 129.12433],
 '서울': [108, 37.57142, 126.9658],
 '인천': [112, 37.47772, 126.6249],
 '원주': [114, 37.33749, 127.94659],
 '울릉도': [115, 37.48129, 130.89864],
 '수원': [119, 37.25746, 126.983],
 '영월': [121, 37.18126, 128.45743],
 '충주': [127, 36.97045, 127.9525],
 '서산': [129, 36.77658, 126.4939],
 '울진': [130, 36.99176, 129.41278],
 '청주': [131, 36.63924, 127.44066],
 '대전': [133, 36.37198, 127.37211],
 '추풍령': [135, 36.22025, 127.99458],
 '안동': [136, 36.57293, 128.70734],
 '상주': [137, 36.40837, 128.15741],
 '포항': [138, 36.03201, 129.38002],
 '군산': [140, 36.0053, 126.76135],
 '대구': [143, 35.87797, 128.65295],
 '전주': [146, 35.84092, 127.1

In [30]:
def get_2week_info(pointTM, place, num):
    endDate = pointTM[:-4]
    hr = pointTM[-4:-2]
    #print(endDate)
    startDate = date.fromisoformat(endDate[:4]+'-'+endDate[4:6]+'-'+endDate[6:]) - timedelta(weeks=2)
    startDate = startDate.isoformat()
    startDate = ''.join(startDate.split('-'))
    print(endDate, hr)
    url = 'http://apis.data.go.kr/1360000/AsosHourlyInfoService/getWthrDataList'
    params ={'serviceKey' : 'XFvzJWgUESMIABoVtfrlH/5UAPe0XlfdQL4aWwGW3PRP7nglPl6l5gbtu9YikPPPpECveIjH0YwuXi9ZccGTcA==', 'pageNo' : '1', 'numOfRows' : '999', 'dataType' : 'XML', 'dataCd' : 'ASOS', 'dateCd' : 'HR', 'startDt' : startDate, 'startHh' : hr, 'endDt' : endDate, 'endHh' : hr, 'stnIds' : place }

    response = requests.get(url, params=params)
    #print(response.content)
    text = response.text

    root = ET.fromstring(response.text)
    result=[]

    for node in root.iter('item'):
        rows = defaultdict(list)
        for i in node:
            #print(i.tag, i.text)
            #print('asdf')
            rows[i.tag] = i.text
        rows['label'] = 0
        result.append(rows)
    #print(result)
    result_a = []
    for i, j in enumerate(result):
        if i == len(result)-1:
            j['label']=1
        sa=dict(j)
        result_a.append(sa)


    sd = pd.DataFrame(result_a, columns=result_a[0].keys())
    print(sd)

In [41]:
# 모든 산불난 사건

#len(df_tm_place)
for i in range(10):
    print(list(df_tm_place.iloc[i]))
    # 만약 앞에 지역이 항목에 없으면 뒤에 지역으로 찾고 그것도 없으면 주변 가까운 지역 찾아서 가져옴
    if str(df_tm_place.iloc[i][1]) in point_dict.keys():
        get_2week_info(str(df_tm_place.iloc[i][0]),point_dict[str(df_tm_place.iloc[i][1])][0], i)
    elif str(df_tm_place.iloc[i][2]) in point_dict.keys():
        get_2week_info(str(df_tm_place.iloc[i][0]),point_dict[str(df_tm_place.iloc[i][2])][0], i)
    else:
        # 먼저 모든 지점까지의 거리를 구함
        # 해당 지점의 위도 경도
        try:
            point_lat=get_lat(str(df_tm_place.iloc[i][1]) +' '+ str(df_tm_place.iloc[i][2])+'구')
            for_compare_distance = [] 
            for k in point_dict.keys():
                for_compare_distance.append([dist((float(point_lat['lat']),float(point_lat['lng'])),(point_dict[k][1],point_dict[k][2])), k])
            print(for_compare_distance)
        except:
            point_lat=get_lat(str(df_tm_place.iloc[i][1]) +' '+ str(df_tm_place.iloc[i][2])+'군')
            for_compare_distance = [] 
            for k in point_dict.keys():
                for_compare_distance.append([dist((float(point_lat['lat']),float(point_lat['lng'])),(point_dict[k][1],point_dict[k][2])), k])
            print(for_compare_distance)
            #우선 여기 프린트만 해놨는데 이거 수정해서 get 함수 쓰도록 수정 바람
#     try:
#         get_2week_info(str(df_tm_place.iloc[i][0]),point_code[str(df_tm_place.iloc[i][2])], i)
#     except:
#         continue

[201406191500, '강원', '인제']
20140619 15
                   tm rnum stnId stnNm    ta taQcflg    rn rnQcflg   ws  \
0    2014-06-05 15:00    1   211    인제  21.7       0  None    None  3.5   
1    2014-06-05 16:00    2   211    인제  22.6       0  None    None  4.1   
2    2014-06-05 17:00    3   211    인제  22.2       0  None    None  4.6   
3    2014-06-05 18:00    4   211    인제  20.7       0  None    None  4.2   
4    2014-06-05 19:00    5   211    인제  19.9       0  None    None  3.0   
..                ...  ...   ...   ...   ...     ...   ...     ...  ...   
332  2014-06-19 11:00  333   211    인제  22.4       0  None       9  2.8   
333  2014-06-19 12:00  334   211    인제  23.3       0  None       9  4.2   
334  2014-06-19 13:00  335   211    인제  22.8       0  None       9  3.3   
335  2014-06-19 14:00  336   211    인제  23.3       0  None       9  2.8   
336  2014-06-19 15:00  337   211    인제  23.6       0  None       9  1.9   

    wsQcflg  ...    vs gndSttCd dmstMtphNo    ts tsQcflg m00

                   tm rnum stnId stnNm    ta taQcflg    rn rnQcflg   ws  \
0    2014-05-27 17:00    1   114    원주  29.7       0  None    None  4.2   
1    2014-05-27 18:00    2   114    원주  28.6       0  None    None  3.6   
2    2014-05-27 19:00    3   114    원주  27.3       0  None    None  2.4   
3    2014-05-27 20:00    4   114    원주  26.2       0  None    None  2.6   
4    2014-05-27 21:00    5   114    원주  25.4       0  None    None  2.8   
..                ...  ...   ...   ...   ...     ...   ...     ...  ...   
332  2014-06-10 13:00  333   114    원주  22.5       0   1.5       0  2.7   
333  2014-06-10 14:00  334   114    원주  24.3       0   0.0       0  3.7   
334  2014-06-10 15:00  335   114    원주  19.9       0   3.5       0  1.0   
335  2014-06-10 16:00  336   114    원주  22.0       0   0.0       0  1.7   
336  2014-06-10 17:00  337   114    원주  22.3       0  None       9  2.0   

    wsQcflg  ...    vs gndSttCd dmstMtphNo    ts tsQcflg m005Te m01Te m02Te  \
0         0  ...  18

[[1.5971143685835723, '속초'], [1.466727189183662, '북춘천'], [1.8815719636453423, '철원'], [1.8671874253961536, '동두천'], [2.086303056844128, '파주'], [1.0534594257292655, '대관령'], [1.4367853888634974, '춘천'], [3.9654019831949316, '백령도'], [1.2154586119461581, '북강릉'], [1.1782761823258385, '강릉'], [1.082740293290782, '동해'], [1.7454251783913197, '서울'], [2.0035427482150294, '인천'], [0.8476745923176016, '원주'], [2.5806779691055097, '울릉도'], [1.5875914605969073, '수원'], [0.5236255166735838, '영월'], [0.5901410972247237, '충주'], [1.9626314042056479, '서산'], [1.0163408393147462, '울진'], [1.0124323784467217, '청주'], [1.1179313166860587, '대전'], [0.6335630968724514, '추풍령'], [0.2681513946961917, '안동'], [0.3866154517246791, '상주'], [1.1184523002212474, '포항'], [1.8130061345619886, '군산'], [0.8049328938294611, '대구'], [1.5656527942604568, '전주'], [1.3905916220962835, '울산'], [1.4922872312515836, '창원'], [2.154584796053253, '광주'], [1.6574349234285284, '부산'], [1.812326578727071, '통영'], [2.7708461640309334, '목포'], [2.04633427241258